# Import the libraries

In [1]:
from rdflib import Graph, Literal, term, URIRef, RDF
from rdflib.namespace import Namespace, NamespaceManager
import pyshacl
import re
import os
from rdflib.namespace import Namespace, NamespaceManager
from NLP2SHACL import write_shacl

# Load the graph

In [63]:
# Load the data graph
data_graph = Graph()
data_graph.parse("InstanceModel/brick_model.ttl", format="turtle")

<Graph identifier=N9e26b9536b084a76ac65380bccfa70ba (<class 'rdflib.graph.Graph'>)>

# Validation Function

In [64]:
def validate_graph(data_graph, shape_graph):
    shapes_file_format = 'turtle'
    data_file_format = 'turtle'

    res = pyshacl.validate(data_graph, shacl_graph = shape_graph,
                                     data_graph_format = data_file_format,
                                     shacl_graph_format = shapes_file_format,
                                     inference='both')

    return res


### Load the invalid data graph

In [85]:
data_graph_invalid = Graph()
data_graph_invalid.parse("InstanceModel/brick_model_invalid.ttl", format="turtle")

<Graph identifier=Nfc92ed0da0a14cefb757c59c38f5f2ab (<class 'rdflib.graph.Graph'>)>

# Brick VAV Example (point count)
### R1 : Every VAV should hasPoint at least 2.

## Using NLP2SHACL tool

In [76]:
# Generate the SHACL
requirement_txt= """
   R1 : Every VAV should hasPoint at least 2.
   """

shacl_txt = write_shacl(requirement_txt)

print('SHACL text generated:\n {0}'.format(shacl_txt))

SHACL text generated:
 
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix inst: <http://linkedbuildingdata.net/ifc/resources20201208_005325/> .
@prefix brick: <https://brickschema.org/schema/Brick#> .
inst:R1 a sh:NodeShape ;
    sh:targetClass brick:VAV ;

sh:property [
sh:path brick:hasPoint ;
 sh:minCount 2 ;
] .


## Run the SHACL against the data graph

In [75]:
# Load the generated shacl text and run validation
shape_graph = shacl_txt
res = validate_graph(data_graph, shape_graph)

print(res[2])

Validation Report
Conforms: True



# Brick HVAC_Zone Example

Each HVAC_Zone should by fed by at least 1 VAV

In [81]:
# Generate the SHACL
requirement_txt= """
   R3 : Every HVAC_Zone should feeds by at least 1 VAV.
   """

shacl_txt = write_shacl(requirement_txt)

print('SHACL text generated:\n {0}'.format(shacl_txt))

SHACL text generated:
 
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix inst: <http://linkedbuildingdata.net/ifc/resources20201208_005325/> .
@prefix brick: <https://brickschema.org/schema/Brick#> .
inst:R3 a sh:NodeShape ;
    sh:targetClass brick:HVAC_Zone ;
    sh:property [
    sh:path [sh:inversePath brick:feeds] ;
sh:minCount 1 ;
	sh:class brick:VAV ;
] .


In [82]:
# Load the generated shacl text and run validation
shape_graph = shacl_txt
res = validate_graph(data_graph, shape_graph)

print(res[2])

Validation Report
Conforms: True



# Brick VAV Point Type Example

### R2 : Every VAV should hasPoint at least 1 Zone_Air_Temperature_Sensor and hasPoint at least 1 Zone_Air_Temperature_Setpoint.

In [ ]:
# Generate the SHACL
requirement_txt= """
   R2 : Every VAV should hasPoint at least 1 Zone_Air_Temperature_Sensor and hasPoint at least 1 Zone_Air_Temperature_Setpoint.
   """

shacl_txt = write_shacl(requirement_txt)

print('SHACL text generated:\n {0}'.format(shacl_txt))

SHACL text generated:
 
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix inst: <http://linkedbuildingdata.net/ifc/resources20201208_005325/> .
@prefix brick: <https://brickschema.org/schema/Brick#> .
inst:R2 a sh:NodeShape ;
    sh:targetClass brick:VAV ;

sh:property [
sh:path brick:hasPoint ;
 sh:minCount 1 ;
	sh:class brick:Zone_Air_Temperature_Sensor ;] ;

sh:property [
sh:path brick:hasPoint ;
 sh:minCount 1 ;
	sh:class brick:Zone_Air_Temperature_Setpoint ;
] .


In [ ]:
# Load the generated shacl text and run validation

shape_graph = shacl_txt
res = validate_graph(data_graph, shape_graph)

print(res[2])

Validation Report
Conforms: False
Results (2):
Constraint Violation in ClassConstraintComponent (http://www.w3.org/ns/shacl#ClassConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:class brick:Zone_Air_Temperature_Sensor ; sh:minCount Literal("1", datatype=xsd:integer) ; sh:path brick:hasPoint ]
	Focus Node: inst:vav1
	Value Node: inst:setpoint1
	Result Path: brick:hasPoint
	Message: Value does not have class brick:Zone_Air_Temperature_Sensor
Constraint Violation in ClassConstraintComponent (http://www.w3.org/ns/shacl#ClassConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:class brick:Zone_Air_Temperature_Setpoint ; sh:minCount Literal("1", datatype=xsd:integer) ; sh:path brick:hasPoint ]
	Focus Node: inst:vav1
	Value Node: inst:sensor1
	Result Path: brick:hasPoint
	Message: Value does not have class brick:Zone_Air_Temperature_Setpoint



# BOT GUID Example
Requirement:
Each bot:Space must have one and only guid and that guid must be a string. 

In [ ]:
# Load the Shape File
shape_graph = Graph()
shape_graph.parse("Shape/ifc_guid.ttl", format="turtle")

In [ ]:
res = validate_graph(data_graph, shape_graph)
res[0]